In [1]:
import sys
import os
sys.path.append(os.path.abspath("/home/italolanza/workspace/TG/runtimes/nnom/"))
print(f'Path: {sys.path}')

from tensorflow.keras import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.models import load_model, save_model
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import numpy as np
from scripts.nnom_utils import *

Path: ['/home/italolanza/workspace/TG/graduation-project/notebooks', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/italolanza/.local/lib/python3.9/site-packages', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/home/italolanza/.local/lib/python3.9/site-packages/IPython/extensions', '/home/italolanza/.ipython', '/home/italolanza/workspace/TG/runtimes/nnom']


2022-04-12 18:24:04.336587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 18:24:04.342184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 18:24:04.342463: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
DATASET_DIR = '/home/italolanza/workspace/TG/dataset/'
DATASET_NAME = 'dataset_completo.csv' 
DATASET_PATH = DATASET_DIR+DATASET_NAME
MODEL_DIR = '/home/italolanza/workspace/TG/graduation-project/models/tf_model/'
NNOM_MODEL_DIR = '/home/italolanza/workspace/TG/graduation-project/models/nnom/'

collum_name = ['Tacom_1f0', 'Tacom_2f0', 'Tacom_3f0', 'Tacom_kurtosis', 'Tacom_entropy', 'Aceler_Underhang_X_1f0', 'Aceler_Underhang_X_2f0', 'Aceler_Underhang_X_3f0', 'Aceler_Underhang_X_kurtosis', 'Aceler_Underhang_X_entropy', 'Aceler_Underhang_Y_1f0', 'Aceler_Underhang_Y_2f0', 'Aceler_Underhang_Y_3f0', 'Aceler_Underhang_Y_kurtosis', 'Aceler_Underhang_Y_entropy', 'Aceler_Underhang_Z_1f0', 'Aceler_Underhang_Z_2f0', 'Aceler_Underhang_Z_3f0', 'Aceler_Underhang_Z_kurtosis', 'Aceler_Underhang_Z_entropy', 'Aceler_Overhang_X_1f0', 'Aceler_Overhang_X_2f0', 'Aceler_Overhang_X_3f0', 'Aceler_Overhang_X_kurtosis', 'Aceler_Overhang_X_entropy', 'Aceler_Overhang_Y_1f0', 'Aceler_Overhang_Y_2f0', 'Aceler_Overhang_Y_3f0', 'Aceler_Overhang_Y_kurtosis', 'Aceler_Overhang_Y_entropy', 'Aceler_Overhang_Z_1f0', 'Aceler_Overhang_Z_2f0', 'Aceler_Overhang_Z_3f0', 'Aceler_Overhang_Z_kurtosis', 'Aceler_Overhang_Z_entropy', 'Audio_1f0', 'Audio_2f0', 'Audio_3f0', 'Audio_kurtosis', 'Audio_entropy', 'Class']
all_features = ['Tacom_1f0', 'Tacom_2f0', 'Tacom_3f0', 'Tacom_kurtosis', 'Tacom_entropy', 'Aceler_Underhang_X_1f0', 'Aceler_Underhang_X_2f0', 'Aceler_Underhang_X_3f0', 'Aceler_Underhang_X_kurtosis', 'Aceler_Underhang_X_entropy', 'Aceler_Underhang_Y_1f0', 'Aceler_Underhang_Y_2f0', 'Aceler_Underhang_Y_3f0', 'Aceler_Underhang_Y_kurtosis', 'Aceler_Underhang_Y_entropy', 'Aceler_Underhang_Z_1f0', 'Aceler_Underhang_Z_2f0', 'Aceler_Underhang_Z_3f0', 'Aceler_Underhang_Z_kurtosis', 'Aceler_Underhang_Z_entropy', 'Aceler_Overhang_X_1f0', 'Aceler_Overhang_X_2f0', 'Aceler_Overhang_X_3f0', 'Aceler_Overhang_X_kurtosis', 'Aceler_Overhang_X_entropy', 'Aceler_Overhang_Y_1f0', 'Aceler_Overhang_Y_2f0', 'Aceler_Overhang_Y_3f0', 'Aceler_Overhang_Y_kurtosis', 'Aceler_Overhang_Y_entropy', 'Aceler_Overhang_Z_1f0', 'Aceler_Overhang_Z_2f0', 'Aceler_Overhang_Z_3f0', 'Aceler_Overhang_Z_kurtosis', 'Aceler_Overhang_Z_entropy', 'Audio_1f0', 'Audio_2f0', 'Audio_3f0', 'Audio_kurtosis', 'Audio_entropy']

In [3]:
# load dataset data
dataset = pd.read_csv(DATASET_PATH, names=collum_name)
dataset_x = dataset[all_features].values
dataset_y = dataset['Class'].values.astype(np.uint8)
x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, shuffle=True)

In [4]:
# load keras saved model
# -------- generate weights.h (NNoM model) ----------
model = load_model(MODEL_DIR)

# generate binary dataset for NNoM validation
generate_test_bin(x_test, y_test, name=DATASET_DIR+'test_data_nnom.bin')

# generate NNoM model, x_test is the calibration dataset used in quantisation process
generate_model(model,  x_test, name=NNOM_MODEL_DIR+"weights.h", quantize_method='max_min')



2022-04-12 18:24:20.332466: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-12 18:24:20.333493: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 18:24:20.333889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 18:24:20.334224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

binary test file generated: /home/italolanza/workspace/TG/dataset/test_data_nnom.bin
test data length: 586
dense_input max value: 925977.8520085523 min value: -20.265629941305107 dec bit -13
dense max value: 283333.4 min value: 0.0 dec bit -12
dense_1 max value: 1.0 min value: 0.0 dec bit 7
shift list {'dense_input': -13, 'dense': -12, 'dense_1': 7}
weights for layer dense
  weight: dense/kernel:0
  original shape:  (40, 32)
  dec bit 4
  reshape to: (1280,)
  bias:  dense/bias:0
  original shape:  (32,)
  dec bit 6
  new dec bit 4
  reshape to: (32,)
weights for layer dense_1
  weight: dense_1/kernel:0
  original shape:  (32, 6)
  dec bit 6
  reshape to: (192,)
  bias:  dense_1/bias:0
  original shape:  (6,)
  dec bit 6
  new dec bit 6
  reshape to: (6,)


In [ ]:
# evaluate in Keras (for comparision)
scores = evaluate_model(model, x_test, y_test)

In [ ]:
# --------- for test in CI ----------
os.walk(NNOM_MODEL_DIR)
# build NNoM
os.system("scons")

# do inference using NNoM
cmd = ".\mnist.exe" if 'win' in sys.platform else "./mnist"
os.system(cmd)
try:
    # get NNoM results
    result = np.genfromtxt('result.csv', delimiter=',', dtype=np.int, skip_header=1)
    result = result[:,0]        # the first column is the label, the second is the probability
    label = y_test_original[:len(y_test)].flatten()     # use the original numerical label
    acc = np.sum(result == label).astype('float32')/len(result)
    if (acc > 0.5):
        print("Top 1 Accuracy on Keras %.2f%%" %(scores[1]*100))
        print("Top 1 Accuracy on NNoM  %.2f%%" %(acc *100))
        return 0
    else:
        raise Exception('test failed, accuracy is %.1f%% < 80%%' % (acc * 100.0))
except:
    raise Exception('could not perform the test with NNoM')